In [ ]:
import pandas as pd
import pymongo
import os
import sys
import inspect
import datetime

In [ ]:
# Database
client = pymongo.MongoClient(input())
database = client['bts_advisor']
collection = database['statcast']

In [ ]:
# Put in a DataFrame
today = datetime.date(2021, 8, 18)
orig_df = pd.DataFrame(collection.find())
statcast_df = orig_df[orig_df['game_date'] < today.strftime('%Y-%m-%d')]
statcast_df

In [ ]:
base_directory = os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))))
base_directory

In [ ]:
sys.path.insert(0, base_directory)
from app import calculate_hit_pct

In [ ]:
# Hit %
hit_pct_df = calculate_hit_pct(statcast_df)
hit_pct_df

In [ ]:
# Hit % (last 7 days)
seven_days_ago = today - datetime.timedelta(days=7)
hit_pct_7_df = calculate_hit_pct(statcast_df[statcast_df['game_date'] >= seven_days_ago.strftime('%Y-%m-%d')])
hit_pct_7_df

In [ ]:
# H / PA vs Relievers
relievers_df = statcast_df[statcast_df['starter_flg'] == False]
xBA_vs_bullpen_df = relievers_df.groupby('batter')['hit'].mean().reset_index().rename({'batter': 'player_id', 'hit': 'H_per_PA'}, axis=1).sort_values(by='H_per_PA', ascending=False)
xBA_vs_bullpen_df

In [ ]:
# H / PA vs Pitcher Hand
xBA_vs_pitcher_hand_df = statcast_df.groupby(['batter', 'pitcher_handedness'])['hit'].mean().reset_index().rename({'batter': 'player_id', 'pitcher_handedness': 'vs', 'hit': 'H_per_PA'}, axis=1).sort_values(by='H_per_PA', ascending=False)
xBA_vs_pitcher_hand_df

In [ ]:
# H / PA vs Batter Hand
xBA_vs_batter_hand_df = statcast_df.groupby(['pitcher', 'batter_handedness'])['hit'].mean().reset_index().rename({'picher': 'player_id', 'batter_handedness': 'vs', 'hit': 'H_per_PA'}, axis=1).sort_values(by='H_per_PA')
xBA_vs_batter_hand_df

In [ ]:
# Starters
starters_list = orig_df[(orig_df['game_date'] == today.strftime('%Y-%m-%d')) & (orig_df['starter_flg'] == True)]['pitcher'].unique().tolist()
starters_list

In [ ]:
# Merge data
merge1 = pd.merge(hit_pct_df, hit_pct_7_df, on=['player_id', 'player_name'], suffixes=('', '7'))
merge2 = pd.merge(merge1, xBA_vs_bullpen_df, on='player_id').rename({'H_per_PA': 'H / PA (BP)'}, axis=1)
merge2

In [ ]:
def calculate_hit_per_pa(statcast_df):
    start_time = time.time() # Start timer

    #batter_vs_pitcher_hand_df = statcast_df.groupby(['batter', 'pitcher_handedness'])['hit'].mean().reset_index().rename({'batter': 'player_id', 'pitcher_handedness': 'vs', 'hit': 'H_per_PA'}, axis=1)
    batter_vs_pitcher_hand_df = statcast_df.pivot_table(values='hit', index='batter', columns='pitcher_handedness').reset_index().rename({'batter': 'player_id'}, axis=1)

    batter_vs_relievers_df = statcast_df[statcast_df['starter_flg'] == False].groupby('batter')['hit'].mean().reset_index().rename({'batter': 'player_id', 'hit': 'H_per_PA'}, axis=1).sort_values(by='H_per_PA', ascending=False)

    pitcher_vs_batter_hand_df = statcast_df[statcast_df['pitcher'].isin(starters_list)].groupby(['pitcher', 'batter_handedness'])['hit'].mean().reset_index().rename({'pitcher': 'player_id', 'batter_handedness': 'vs', 'hit': 'H_per_PA'}, axis=1)

    #stop_timer('calculate_hit_per_pa()', start_time) # Stop timer
    return batter_vs_pitcher_hand_df, batter_vs_relievers_df, pitcher_vs_batter_hand_df
calculate_hit_per_pa(statcast_df)

In [ ]:
import time